<a href="https://colab.research.google.com/github/addalasaikishore/kishore_INFO5731_SPRING2021/blob/main/In_class_exercise_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The ninth in-class-exercise (20 points in total, 4/16/2021)**

The purpose of the exercise is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training. 

The dataset can be download from here: https://github.com/unt-iialab/info5731_spring2021/blob/main/class_exercises/exercise09_datacollection.zip. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data. 

Algorithms:

(1) MultinominalNB

(2) SVM 

(3) KNN 

(4) Decision tree

(5) Random Forest

(6) XGBoost

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison 

(4) F-1 score

# **Read and Formating Trainig Dataset**

In [2]:
import pandas as pd
Data = open('stsa-train.txt').read()
Labels, Text = [], []
for i, line in enumerate(Data.split("\n")):
    content = line.split(' ')
    Labels.append(content[0])
    Text.append(" ".join(content[1:]))

# **Read and Formating Testing Dataset**

In [3]:
Testing_data = open('stsa-test.txt').read()
Test_labels, Test_text = [], []
for i, line in enumerate(Testing_data.split("\n")):
    content = line.split(' ')
    Test_labels.append(content[0])
    Test_text.append(" ".join(content[1:]))

# **Creating Dataframe**

In [8]:
df = pd.DataFrame (list(zip(Text, Labels)) , columns = ['Raw Text', 'Labels'])
Test_df = pd.DataFrame (list(zip(Test_text, Test_labels)) , columns = ['Raw Text', 'Labels'])
Test_df = Test_df.dropna()

In [9]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> D

Download which package (l=list; x=cancel)?
  Identifier> ALL
    Error loading ALL: Package 'ALL' not found in index

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> all
Command 'all' unrecognized

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data.

True

# **Data Cleaning**

In [10]:
import re
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.stem import PorterStemmer
from textblob import Word
nltk.download('wordnet')
st = PorterStemmer()
nltk.download('punkt')
stop = stopwords.words('english')
df['Lower Case'] = df['Raw Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['Removal of Punctuation'] = df['Lower Case'].str.replace('[^\w\s]','')
df['Removal of Special Characters'] = df['Removal of Punctuation'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))
df['Stopwords Removal'] = df['Removal of Punctuation'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['Tokenization'] = df['Stopwords Removal'].apply(lambda x: TextBlob(x).words)
df['After Lemmatization'] = df['Tokenization'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
Test_df['Lower Case'] = Test_df['Raw Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
Test_df['Removal of Punctuation'] = Test_df['Lower Case'].str.replace('[^\w\s]','')
Test_df['Removal of Special Characters'] = Test_df['Removal of Punctuation'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))
Test_df['Stopwords Removal'] = Test_df['Removal of Punctuation'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
Test_df['Tokenization'] = Test_df['Stopwords Removal'].apply(lambda x: TextBlob(x).words)
Test_df['After Lemmatization'] = Test_df['Tokenization'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))

# **Data Transformation**

In [25]:
from sklearn import model_selection, preprocessing, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(analyzer='word')
tfidf_vec.fit(df['After Lemmatization'])
x_tfidf =  tfidf_vec.transform(df['After Lemmatization'])

In [26]:
vec_test = TfidfVectorizer(analyzer='word', vocabulary = tfidf_vect.vocabulary_)
vec_test.fit(Test_df['After Lemmatization'])
xtest = vec_test.transform(Test_df['After Lemmatization'])
test_y = Test_df['Labels']

# **Splitting Data into Training and Testing**

In [27]:
from sklearn import model_selection, preprocessing, naive_bayes, metrics, svm
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(x_tfidf, df['Labels'].values,test_size=0.2)

In [28]:
from sklearn import metrics
def get_metrics(predictions, test_data_y):
  accuracy = metrics.accuracy_score(predictions, test_data_y)
  precision = metrics.precision_score(predictions, test_data_y, pos_label='positive', average='micro')
  recall = metrics.recall_score(predictions, test_data_y, pos_label='positive', average='micro')
  f1 = metrics.f1_score(predictions, test_data_y, pos_label='positive', average='micro')
  return accuracy, precision, recall, f1

In [29]:
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
def cross_validation_score(modelName, x, y):
  scoring = 'accuracy'
  kfold = KFold(10, random_state = 7,shuffle=True)
  cross_val = cross_val_score(modelName, x, y, cv=kfold).mean()
  return cross_val

# **Training And Evaluating Models**

**Navie Bayes - Validation Metrics**

In [30]:
naive_bayes_model = naive_bayes.MultinomialNB()
naive_bayes_model.fit(train_x, train_y)
naive_bayes_predictions_validation_data = naive_bayes_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(naive_bayes_predictions_validation_data, valid_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))

Accuracy is 0.7740072202166065
Precision is 0.7740072202166065
Recall is 0.7740072202166065
F1 is 0.7740072202166065


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


**Cross Validation Score**

In [43]:
scores_mean = cross_validation_score(naive_bayes_model, valid_x, valid_y)
print('Navie Bayes Cross Validation Score  {0}'.format(scores_mean))

Navie Bayes Cross Validation Score  0.7199145031800647


**Testing Metrics**

In [42]:
naive_bayes_predictions_test_data = naive_bayes_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(naive_bayes_predictions_test_data, test_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1  {3}".format(accuracy, precision, recall, f1))

Accuracy is 0.7881448957189902
Precision is 0.7881448957189902
Recall is 0.7881448957189902
F1  0.78814489571899


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


**Cross Validation Score - Testing**

In [41]:
Nb_test_scores_mean = cross_validation_score(naive_bayes_model, xtest, test_y)
print('Navie Bayes Cross Validation Score {0}'.format(Nb_test_scores_mean))

Navie Bayes Cross Validation Score 0.7371044256290158


**SVM - Validation Metrics**

In [36]:
svm_model = svm.SVC()
svm_model.fit(train_x, train_y)
svm_predict_validation_data = svm_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(svm_predict_validation_data, valid_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))

Accuracy is 0.7725631768953068
Precision is 0.7725631768953068
Recall is 0.7725631768953068
F1 is 0.7725631768953068


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


**Cross Validation Score**

In [40]:
svm_scores_mean = cross_validation_score(svm_model, valid_x, valid_y)
print('SVM Cross Validation Score  {0}'.format(svm_scores_mean))

SVM Cross Validation Score  0.7126681263684704


**Testing Metrics**

In [38]:
svm_predictions_test_data = svm_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(svm_predictions_test_data, test_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))

Accuracy is 0.7859495060373216
Precision is 0.7859495060373216
Recall is 0.7859495060373216
F1 is 0.7859495060373216


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


**Cross Validation Score - Testing**

In [39]:
svm_test_scores_mean = cross_validation_score(svm_model, xtest, test_y)
print('SVM Cross Validation Score  {0}'.format(svm_test_scores_mean))

SVM Cross Validation Score  0.7239086050561461


**KNN - Validation Metrics**

In [45]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(n_neighbors = 15)
knn_model.fit(train_x, train_y)
knn_predict_valid_data = knn_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(knn_predict_valid_data, valid_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1  {3}".format(accuracy, precision, recall, f1))

Accuracy is 0.6981949458483755
Precision is 0.6981949458483755
Recall is 0.6981949458483755
F1  0.6981949458483755


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


**Cross Validation Score**

In [47]:
knn_score_mean = cross_validation_score(knn_model, valid_x, valid_y)
print('KNN Cross Validation Score  {0}'.format(knn_score_mean))

KNN Cross Validation Score  0.6823271817328745


**Testing Metrics**

In [49]:
knn_predict_test_data = knn_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(knn_predict_test_data, test_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1  {3}".format(accuracy, precision, recall, f1))

Accuracy is 0.6942919868276619
Precision is 0.6942919868276619
Recall is 0.6942919868276619
F1  0.6942919868276619


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


**Cross Validation Score - Testing**

In [51]:
knn_test_score_mean = cross_validation_score(knn_model, xtest, test_y)
print('KNN Cross Validation Score {0}'.format(knn_test_score_mean))

KNN Cross Validation Score 0.6361376328589443


**Decision Tree - Validation Metrics**

In [54]:
from sklearn.tree import DecisionTreeClassifier
Dt_model = DecisionTreeClassifier()
Dt_model.fit(train_x, train_y)
Dt_predictions_valid_data = Dt_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(Dt_predictions_valid_data, valid_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1  {3}".format(accuracy, precision, recall, f1))

Accuracy is 0.6433212996389892
Precision is 0.6433212996389892
Recall is 0.6433212996389892
F1  0.6433212996389892


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


**Cross Validation Score**

In [59]:
Dt_score_mean = cross_validation_score(Dt_model, valid_x, valid_y)
print('Decision Tree Cross Validation Score {0}'.format(Dt_score_mean))

Decision Tree Cross Validation Score 0.6151548326556147


**Testing Metrics**

In [61]:
Dt_predict_test_data = Dt_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(Dt_predict_test_data, test_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))

Accuracy is 0.6597145993413831
Precision is 0.6597145993413831
Recall is 0.6597145993413831
F1 is 0.6597145993413831


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


**Cross Validation Score - Testing**

In [62]:
Dt_test_score_mean = cross_validation_score(Dt_model, xtest, test_y)
print('Decision Tree Cross Validation Score is {0}'.format(Dt_test_score_mean))

Decision Tree Cross Validation Score is 0.6267579415120399


**Random Forest - Validation Metrics**

In [63]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier()
rf_model.fit(train_x, train_y)
rf_predict_valid_data = rf_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(rf_predict_valid_data, valid_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))

Accuracy is 0.7393501805054151
Precision is 0.7393501805054151
Recall is 0.7393501805054151
F1 is 0.7393501805054151


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


**Cross Validation Score**

In [64]:
rf_score_mean = cross_validation_score(rf_model, valid_x, valid_y)
print('Random Forest Cross Validation Score is {0}'.format(rf_score_mean))

Random Forest Cross Validation Score is 0.6599363987071213


**Testing Metrics**

In [65]:
rf_predict_test_data = rf_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(rf_predict_test_data, test_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))

Accuracy is 0.7338090010976949
Precision is 0.7338090010976949
Recall is 0.7338090010976949
F1 is 0.7338090010976949


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


**Cross Validation Score - Testing**

In [66]:
rf_test_score_mean = cross_validation_score(rf_model, xtest, test_y)
print('Random Forest Cross Validation Score  {0}'.format(rf_test_score_mean))

Random Forest Cross Validation Score  0.6728877679697352


**XG Boost - Validation Metrics**

In [68]:
from xgboost import XGBClassifier
xg_model = XGBClassifier()
xg_model.fit(train_x, train_y)
xg_predict_valid_data = xg_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(xg_predict_valid_data, valid_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))

Accuracy is 0.6512635379061372
Precision is 0.6512635379061372
Recall is 0.6512635379061372
F1 is 0.6512635379061372


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


**Cross Validation Score**

In [69]:
xg_score_mean = cross_validation_score(XGBClassifier(), valid_x, valid_y)
print('XG Boost Cross Validation Score is {0}'.format(xg_score_mean))

XG Boost Cross Validation Score is 0.6057084766969034


**Testing Metrics**

In [70]:
xg_predict_test_data = xg_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(xg_predict_test_data, test_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))

Accuracy is 0.6383095499451152
Precision is 0.6383095499451152
Recall is 0.6383095499451152
F1 is 0.6383095499451152


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


**Cross Validation Score - Testing**

In [ ]:
xg_test_score_mean = cross_validation_score(XGBClassifier(), xtest, test_y)
print('XG Boost Cross Validation Score is {0}'.format(xg_test_score_mean))

XG Boost Cross Validation Score is 0.6251035849396506


**Accuracies of the models when use test data -**

1. Navie Bayes - 78.86%
2. SVM - 78.70%
3. KNN - 72.94%
4. Decision Tree - 65.14%
5. Random Forest - 74.14%
6. XG Boost - 64.92%